In [ ]:
"""
- 앙상블 모델들은 결정트리(DT)를 기반으로 만들어진 모델들임
- 앙상블 학습모델(분류 및 회귀 모두 사용가능)
  : 랜덤포레스트(RandomForest : RT)
  : 엑스트라 트리(Extra Trees : ET)
  : 그레디언트 부스팅(Gradient Boosting : GD)
  : 히스토그램 기반 그레이언트 부스팅(Histogram-base Gradient Boosting : HGB) *
- 앙상블 학습모델(분류로만 사용 가능한 모델)
  : XGBooting(XGB)
  : LightGBM(LGBM) *

"""

In [1]:
"""
<랜덤포레스트>
- 앙상블 학습 모델의 가장 대표격 모델
- 안정적인 성능으로 널리 사용되고 있음
- 과대적합되는 것을 막아줌 ***
- 검증 및 테스트데이터에서 안정적 성능을 얻을 수 있음

<처리 순서>
- 결정트리를 랜덤하게 만들어 숲을 만듦
- 훈련데이터에서 랜덤하게 샘플 추출
- 사용된 샘플은 다시 훈련데이터에 넣어서 랜덤하게 샘플 훈련 데이터 생성
- 추출한 샘플 훈련데이터는 일부 중복된 값들로 추출될 수 있음

<부트스트랩 샘플링(Bootstrap Sample)>
- 데이터 원본에서 중복을 허용하여 데이터를 샘플링하는 방식
- 원본에서 랜덤하게 샘플을 추출
- 사용이 끝난 샘플을 원본에 다시 반환
- 위의 순서를 반복해 다시 샘플 추출

"""

'\n\n- 앙상블 학습 모델의 가장 대표격 모델\n- 안정적인 성능으로 널리 사용되고 있음\n- 과대적합되는 것을 막아줌 ***\n- 검증 및 테스트데이터에서 안정적 성능을 얻을 수 있음\n\n<\n'

In [32]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [3]:
# 데이터 불러오기
wine = pd.read_csv("./data/08_wine.csv")

# 훈련데이터(input, target) 생성
wine_input = wine[["alcohol","sugar","pH"]].to_numpy() # 2차원
wine_target = wine["class"].to_numpy() # 1차원

# 훈련데이터와 테스트데이터로 분리
train_input, test_input, train_target, test_target = train_test_split(wine_input,wine_target,
                                                                      test_size=0.2,random_state=42)
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [7]:
# 교차검증을 통해 랜덤포레스트 모델 사용
# 모델생성
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# 교차검증
# return_train_score : 훈련 및 검증결과 보여주기
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(scores)

{'fit_time': array([0.15399957, 0.15700006, 0.15499973, 0.16399956, 0.15499997]), 'score_time': array([0.02399993, 0.02399993, 0.02399993, 0.02400017, 0.02499986]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}
0.9973541965122431
0.8905151032797809


In [11]:
# 최종 훈련평가 결과 및 검증결과
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print(train_mean, test_mean)

# 과대적합 발생

0.9973541965122431 0.8905151032797809


In [12]:
# 튜닝
# 특성중요도 확인
# 랜덤포레스트 자체 훈련시키기
rf.fit(train_input,train_target)
print(rf.feature_importances_)
# 특성의 순서 : 알콜, 당도, 농도

[0.23167441 0.50039841 0.26792718]


In [14]:
# 랜덤포레스트만의 특징
# 모델생성
# oob_score(Out of Bag) 샘플링
#    : 교차검증 중 훈련에 참여하지 못한 샘플이 발생할 수 있는데 
#      이러한 샘플을 훈련에 참여시킴으로써 훈련의 정확도 올림
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
# 훈련시키기
rf.fit(train_input,train_target)
print(rf.oob_score_)
print(rf.score(train_input,train_target))
print(rf.score(test_input,test_target))
# 해석
# - oob가 사용된 score 비교시 oob 검증결과(oob_score_)와 테스트 검증결과를 비교
# - 0.89 vs 0.88로 성능은 높지 않지만, 과대적합은 해소

0.8934000384837406
0.996921300750433
0.8892307692307693


In [ ]:
"""
- 랜덤포레스트와 유사
- 기본적으로 100개의 결정트리를 훈련
- 랜덤포레스트와의 차이점
  : 부트스트랩을 지원하지 않음
  : 훈련데이터 전체를 이용
  : 무작위로 트리 분할
- 특성이 많지 않은 경우 랜덤포레스트와 큰 차이 없음
- 랜덤하게 트리를 분할하기 때문에(알아서) 계산 속도 빠름
"""

In [18]:
# 모델생성
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
# 교차검증
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)


# 최종 훈련평가 결과 및 검증결과
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print(train_mean, test_mean)

0.9974503966084433 0.8887848893166506


In [19]:
# 그래디언트 부스팅(Gradient Boosting)
"""
- 깊이(max_depth)가 얕은 결정트리 사용
  : 기본적으로 max_depth는 3
  : 결정트리는 100개 사용
- 기존에 다른 훈련모델의 결과가 좋지 않은 경우 사용해 볼 수 있음
- 기존 훈련모델의 오차를 보완하여 성능을 향상하고자 할 때 사용하면 좋음
- 랜덤포레스트보다 과대적합에 강하며, 일반화에 강함

<단점>
- 순서대로 트리를 추가(랜덤하지 않음)하기 때문에 랜덤포레스트보다 훈련속도 느림
- 느린 속도를 개선한 모델이 '히스토그램 기반 그래디언트 부스팅 모델'임
"""

''

In [22]:
# 모델생성
gd = GradientBoostingClassifier(random_state=42)
# 교차검증
scores = cross_validate(gd, train_input, train_target,
                        return_train_score=True, n_jobs=-1)


# 최종 훈련평가 결과 및 검증결과
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print(train_mean, test_mean)

# 과대적합 해소

0.8881086892152563 0.8720430147331015


In [23]:
# 모델생성
# 하이퍼 파라미터 추가
# n_estimators : 결정트리 개수
# learning_rate : 학습률(기본값 : 0.1, 성능향상시 값을 높이면서 확인)
gd = GradientBoostingClassifier(n_estimators=500,
                                learning_rate=0.2,
                                random_state=42)
# 교차검증
scores = cross_validate(gd, train_input, train_target,
                        return_train_score=True, n_jobs=-1)


# 최종 훈련평가 결과 및 검증결과
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print(train_mean, test_mean)

# 과대적합 발생 -> 하이퍼파라미터 안쓰는게 낫다

0.9464595437171814 0.8780082549788999


In [ ]:
"""
- 그래디언트 부스팅의 느린속도를 개선한 모델
- 인기가 상승하는 모델임
- 속도 빠른 이유
  : 최초 특성을 256개의 구간으로 나누고 시작
  : 트리를 분할 할 때 매우 빠르게 찾을 수 있음
- 장점
  : 하이퍼파라미터의 기본값 만으로 안정적 성능을 얻을 수 있음
- 성능개선은 max_iter(훈련반복횟수)를 통해 테스트 진행가능
"""

In [25]:
# 모델생성
hgb = HistGradientBoostingClassifier(random_state=42,)
# 교차검증
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)


# 최종 훈련평가 결과 및 검증결과
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print(train_mean, test_mean)

# 과대적합 해소

0.9321723946453317 0.8801241948619236


In [30]:
# XGBoost
# 모델생성
# tree_method : 사이킷런에서와 유사한 히스토그램 기반 그래디언트부스팅 사용 옵션
xgb = XGBClassifier(tree_method="hist",random_state=42)
# 교차검증
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)


# 최종 훈련평가 결과 및 검증결과
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print(train_mean, test_mean)


0.9555033709953124 0.8799326275264677


In [ ]:
"""
- 마이크로소프트에서 만든 히스토그램 기반 그래디언트 부스트 패키지
- 훈련 속도 빠름
- 좋은건 다 적용했다 보면 됨
- 인기가 늘어나는 모델
"""

In [33]:
# 모델생성
lgbm = LGBMClassifier(random_state=42)
# 교차검증
scores = cross_validate(lgbm, train_input, train_target,
                        return_train_score=True, n_jobs=-1)


# 최종 훈련평가 결과 및 검증결과
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print(train_mean, test_mean)

0.935828414851749 0.8801251203079884
